<h2 style="text-align: center">Final exam Image Processing </h2>
<h3 style="text-align: center">Sarawin Buakaew ID: 63-010725-2017-2</h3>




In [11]:
import numpy as np
import cv2 as cv
from cv2 import aruco
import os 
import matplotlib.pyplot as plt
import scipy

<h3 style="text-align: center;">
    Camera Calibration with OpenCV
</h3>

In [12]:
chessboard_size = (8,6) # Your chessboard size

# iterative termination criteria, maximum iterationm and epsilon
term_criteria = (cv.TermCriteria_EPS+ cv.TermCriteria_MAX_ITER, 30, 0.001)

In [13]:
# List to store object points and images point from all the images.
obj_points = list() # 3D points in heterogeneous Xi
img_points = list() # 2D points on image xi

# Defining the world coordinates for 3D points Xi
objp = np.zeros((1, chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
prev_img_shape = None

In [14]:
def preprocessing(img) :
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img_gray

In [15]:
image_path = os.getcwd()+'/images/stereo/calibrationdata/'
images = list()
image_size = None
print(image_path)

# Preparing calibration files
for filename in os.listdir(image_path) :
    print(filename)
    img = cv.imread(image_path+'{}'.format(filename))
    images.append(img)

d:\Term1_65\Image_Processing\Image_Processing_github\Final_Exam/images/stereo/calibrationdata/
left-0000.png
left-0001.png
left-0002.png
left-0003.png
left-0004.png
left-0005.png
left-0006.png
left-0007.png
left-0008.png
left-0009.png
left-0010.png
left-0011.png
left-0012.png
left-0013.png
left-0014.png
left-0015.png
left-0016.png
left-0017.png
left-0018.png
left-0019.png
left-0020.png
left-0021.png
left-0022.png
left-0023.png
left-0024.png
left-0025.png
left-0026.png
left-0027.png
left-0028.png
left-0029.png
left-0030.png
left-0031.png
left-0032.png
left-0033.png
left-0034.png
left-0035.png
left-0036.png
left-0037.png
left-0038.png
left-0039.png
left-0040.png
left-0041.png
left-0042.png
left-0043.png
left-0044.png
left-0045.png
left-0046.png
left-0047.png
left-0048.png
left-0049.png
left-0050.png
left-0051.png
left-0052.png
left-0053.png
left-0054.png
left-0055.png
left-0056.png
right-0000.png
right-0001.png
right-0002.png
right-0003.png
right-0004.png
right-0005.png
right-0006.png
ri

In [16]:
for image in images :
    img_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(img_gray, chessboard_size, None) # xi in pixel (20,15)
    image_size = img_gray.shape
    # If found add obj points, image points afterthat refining them
    if ret == True :
        #print('Found')
        obj_points.append(objp) # Add Xi 3D
        
        corners2 = cv.cornerSubPix(img_gray, corners, (11,11), (-1,-1), term_criteria) # Refining xi -> xi in subpixel, xi -> floating point (19.7, 15.1)
        img_points.append(corners2) # Add xi 2D

        #Draw and display the chessboard corners
        img = cv.drawChessboardCorners(image, chessboard_size, corners2, ret)
        cv.imshow('frame', img)
        cv.waitKey(100)

cv.destroyAllWindows()

<h3 style="text-align: center">
    <a href = "https://docs.opencv.org/4.5.3/d9/d0c/group__calib3d.html#ga3207604e4b1a1758aa66acb6ed5aa65d"> calibrateCamera </a>
    is the main function which you can use to calibrate a pinhole camera model <br>
</h3>

In [17]:
# # Start calibration argmin ||xi - PXi||^2 , xi = PXi
print('Calibration')
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(obj_points,img_points, image_size[::-1] , None, None)

Calibration


<h4 style="text-align: center">
    <a href = "https://docs.opencv.org/4.5.3/d9/d0c/group__calib3d.html#ga7a6c4e032c97f03ba747966e6ad862b1"> getOptimalNewCameraMatrix </a>
    The function computes and returns the optimal new camera intrinsic matrix based on the free scaling parameter <br>
</h4>

In [18]:
img = images[0].copy()
h, w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
print(mtx)
print('Optimal K')
print(newcameramtx, roi)

[[726.47630219   0.         466.25758541]
 [  0.         732.0640605  251.997867  ]
 [  0.           0.           1.        ]]
Optimal K
[[728.84155273   0.         473.45067169]
 [  0.         728.33746338 251.73843936]
 [  0.           0.           1.        ]] (1, 6, 861, 468)


<h3 style="text-align: center">
    <a href = "https://docs.opencv.org/4.5.3/d9/d0c/group__calib3d.html#ga69f2545a8b62a6b0fc2ee060dc30559d"> undistort </a>
    The function transforms an image to compensate radial and tangential lens distortion. <br>
</h3>

In [19]:
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
undistored = cv.undistort(img, mtx, dist, None, None)
x,y,w,h = roi   
dst = dst[y:y+h, x:x+w]

In [20]:
mean_error = 0
for i in range(len(obj_points)) :
    reprojected_point,_ = cv.projectPoints(obj_points[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(img_points[i], reprojected_point, cv.NORM_L2)/len(reprojected_point)
    mean_error += error

print("Reprojection error: {}".format(mean_error))

while True :
    cv.imshow('calibrated', dst)
    cv.imshow('undistort', undistored)
    if cv.waitKey(1) & 0xFF == ord('q') : 
        break
cv.destroyAllWindows()

Reprojection error: 10.099807080135967


In [21]:
# #Saving parameters into numpy format
# np.save("./new-camera_params/ret", ret)
# np.save("./new-camera_params/K", mtx)
# np.save("./new-camera_params/dist", dist)
# np.save("./new-camera_params/rvecs", rvecs)
# np.save("./new-camera_params/tvecs", tvecs)

In [22]:
def augmented_image(frame,im_src, pts_src, pts_dst):
    
    # Calculate Homography
    h, status = cv.findHomography(pts_src, pts_dst)

    # Warp source image to destination based on homography
    warped_image = cv.warpPerspective(im_src, h, (frame.shape[1],frame.shape[0]))
            
    # Prepare a mask representing region to copy from the warped image into the original frame.
    mask = np.zeros([frame.shape[0], frame.shape[1]], dtype=np.uint8)
    cv.fillConvexPoly(mask, np.int32(pts_dst), (255, 255, 255), cv.LINE_AA)
    im_out = cv.add(frame, warped_image, mask=cv.bitwise_not(mask))
    im_out = cv.add(im_out, warped_image)
    
    return im_out

In [23]:
sift = cv.SIFT_create()
bf = cv.BFMatcher() 

In [84]:
params_dir = os.getcwd()+'/new-camera_params/' #Calibration -> Parallel image 
print(params_dir)

# Load camera parameters
# ret = np.load(params_dir+'ret.npy')
K = np.load(params_dir+'K.npy')
dist = np.load(params_dir+'dist.npy')
# focal_length = (K[0,0] + K[1,1])/2

print("Camera matrix")
print(K)
print("Len distortion")
print(dist)

# chessboard_size = (8,6) # Your chessboard size
# # iterative termination criteria, maximum iterationm and epsilon
# term_criteria = (cv.TermCriteria_EPS+ cv.TermCriteria_MAX_ITER, 30, 0.001)
# # Defining the world coordinates for 3D points
# objp = np.zeros((1, chessboard_size[0] * chessboard_size[1], 3), np.float32)
# objp[0,:,:2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
# # 3D object for visualization
# axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)
# box = np.float32([[0,0,0], [0,4,0], [4,4,0], [4,0,0],
#                    [0,0,-4],[0,4,-4],[4,4,-4],[4,0,-4] ])

d:\Term1_65\Image_Processing\Image_Processing_github\Final_Exam/new-camera_params/
Camera matrix
[[726.47630219   0.         466.25758541]
 [  0.         732.0640605  251.997867  ]
 [  0.           0.           1.        ]]
Len distortion
[[ 0.02450982  0.03012135  0.00045661  0.01036187 -0.07354287]]


In [85]:
# AruCo_dict = cv.aruco.Dictionary_get(cv.aruco.DICT_4X4_50) # Given Aruco
# AruCo_params = cv.aruco.DetectorParameters_create()
# board = cv.aruco.GridBoard_create(1, 1, 0.182, 0.0075, AruCo_dict)

In [86]:
# def write_text(img, pose, dy, text) :
#     x0 = pose[0]
#     y0 = pose[1]
#     for i, line in enumerate(text.split('\n')) :
#         y = y0 + i*dy
#         cv.putText(img, line, np.int32([x0, y]), cv.FONT_HERSHEY_COMPLEX, 0.75, (50,200,255), 2)

In [92]:
vid = cv.VideoCapture('Final_exam/Dataset-1/left_output-1.avi')
template_img = cv.imread('Templates/Template-1.png') 
# qr = cv.imread('./QR-Code_new.jpg') 
# print(qr.shape)
# im_src_size = qr.shape[:2] 
# src_points = np.float32([[-0.182,-0.240], [im_src_size[1],0],[im_src_size[1], im_src_size[0]] ,[0, im_src_size[0]] ]) 
template_gray = preprocessing(template_img) 

In [93]:
while vid.isOpened() :
    ret,  frame = vid.read()  

    if ret :
        
        query_img = frame
        kernel = np.array([[0,0,0], 
                           [0, 1.5,0],
                           [0,0,0]])  
        filter = cv.filter2D(query_img, -1, kernel) 
        query_gray = preprocessing(filter)
    
        template_kpts, template_desc = sift.detectAndCompute(template_img, None)
        query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
        matches = bf.knnMatch(template_desc, query_desc, k=2)

        good_matches = list()
        good_matches_list = list()
        for m, n in matches :
            if m.distance < 0.7*n.distance :
                good_matches.append(m)
                good_matches_list.append([m])
        

        if len(good_matches) > 8 : 
            src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
            dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

            H, inlier_masks = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 1.2) 
            
            h, w = template_img.shape[:2]
            template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2) 
            transformed_box = cv.perspectiveTransform(template_box, H)

            detected_img = cv.polylines( frame, [np.int32(transformed_box)], True, (0,0,255), 2, cv.LINE_AA)
            drawmatch_img = cv.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)

            # augmented = augmented_image(detected_img, qr, src_points,transformed_box)

            # img = augmented.copy() 
            # markerCorners, markerIds, rejectedCandidates = cv.aruco.detectMarkers(augmented , AruCo_dict, parameters = AruCo_params)
        
            # if len(markerCorners) > 0:
            #     img = cv.aruco.drawDetectedMarkers(augmented , markerCorners)
            #     rvecs, tvecs, points = cv.aruco.estimatePoseSingleMarkers(markerCorners , 0.05, K, dist)
            #     for (rvec, tvec, id, corner) in zip(rvecs, tvecs, markerIds, markerCorners) :
            #         img = cv.drawFrameAxes(augmented , K, dist, rvec, tvec, 0.05)
            #         x = tvec[0,0]
            #         y = tvec[0,1]
            #         z = tvec[0,2]
            #         text = " X :{:.3f} \n Y :{:.3f} \n Z :{:.3f}".format( x, y, z) 
            #         cX = (corner[0,0][0] + corner[0,2][0])/ 2
            #         cY = (corner[0,0][1] + corner[0,2][1])/ 2  
            #         write_text(detected_img, (cX, cY), 30, text) 
            #     ret, brvec, btvec = cv.aruco.estimatePoseBoard(markerCorners, markerIds, board, K, dist, rvecs, tvecs)
            #     if ret :
            #        img = cv.drawFrameAxes(frame, K, dist, brvec, btvec, 0.05)


            cv.imshow('Video exam',detected_img )
           
        if cv.waitKey(1) & 0xFF == ord('q') :
            break
    else :
        break

vid.release  ()
cv.destroyAllWindows ()

In [31]:
# vid = cv.VideoCapture('Final_exam/Dataset-1/left_output-1.avi')
# template_img = cv.imread('Templates/Template-4.png')
# template_gray = preprocessing(template_img)

# # plt.imshow(template_gray)

In [32]:
# sift = cv.SIFT_create()
# bf = cv.BFMatcher()

<h4 style="text-align: center">VidoCapture is OpenVideo</h4>

In [33]:
# while vid.isOpened():
    
#     ret, frame = vid.read()
    
#     if ret :
#         query_img = frame
#         kernel = np.array([[0,0,0],
#                            [0,1.5,0],
#                            [0,0,0]])
#         filter = cv.filter2D(query_img, -1,kernel)
#         query_gray = preprocessing(filter)
#         template_kpts, template_desc = sift.detectAndCompute(template_img, None)
#         query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
#         matches = bf.knnMatch(template_desc, query_desc, k=2)
    
    
#         good_matches = list()
#         good_matches_list = list()
#         for m, n in matches :
#             if m.distance < 0.7*n.distance :
#                 good_matches.append(m)
#                 good_matches_list.append([m])
    
#         if len(good_matches) > 14 :
#             src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
#             dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

#             H, inlier_masks = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 10.0) # H RANSAC
#             # get the bounding box around template image
#             h, w = template_img.shape[:2]
#             template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
#             transformed_box = cv.perspectiveTransform(template_box, H)

#             detected_img = cv.polylines(frame, [np.int32(transformed_box)], True, (0,0,255), 3, cv.LINE_AA)
#             drawmatch_img = cv.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)

#         # detected, drawmatch =  feature_object_detection(template_img, query_gray, frame, 10)
    
#         cv.imshow('Video exam',detected_img)
#         # if cv.waitKey(int(1000/20)) & 0xFF == ord('q') : # int(1000/20) ทำให้ video slowly
#         if cv.waitKey(33) & 0xFF == ord('q'):
#             break
#     else :
#             break

# vid.release()
# cv.destroyAllWindows()